In [47]:
import pickle
import multiprocessing
import os, re, sys
import nltk
import csv
import gensim.models.word2vec as w2v
import glob2
import time
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf_vectorizer = TfidfVectorizer()


In [18]:
songdata_filepath = "songdata.csv"                #simple csv

reddit_filepath = "reddit-top-2.5-million-master/data" #this is a dir, we'll walk through it to create a
                                                  #dict of subreddits and posts

songs_list = []

with open(songdata_filepath, 'rb') as csvfile:
    song_reader = csv.reader(csvfile)
    i = 0
    for row in song_reader:
        if(i == 0):
            next
        else:
            song = {'artist': row[0], 'title': row[1], 'lyric_link': row[2], 'text': row[3]}
            songs_list.append(song)
        i += 1
        

subreddit_list = {}

path = reddit_filepath + '/**'
files = glob2.glob(path)

len(files)   #should be 2,499 subreddits

subreddit_id_to_title = {}

j = 0
for file in files:
    subreddit = file.replace(reddit_filepath + "/", "").replace(".csv", "")
    subreddit_id_to_title[j] = subreddit
    j += 1
    with(open(file, 'rb')) as csvfile:
        reddit_reader = csv.reader(csvfile)
        posts = []
        reddit_reader.next()
        for post in reddit_reader:
            #posts.append({'score': post[1].decode('utf-8'), 'title': post[4].decode('utf-8'), 'text': post[9].decode('utf-8')})
            posts.append({'score': post[1], 'title': post[4], 'text': post[9]})
        subreddit_list[subreddit] = posts



In [16]:
def preprocess(str):
    # remove links
    #str = re.sub(r'http(s)?:\/\/\S*? ', "", str)
    str = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', "", str)
    return str

subreddits_without_titles = []
offset = 2400

for i in range(0, len(subreddit_list) - 1 - offset):
    reddit_corpus = ""
    subreddit = subreddit_list[subreddit_id_to_title[i]]
    for post in subreddit:
        string_ripped_out = post['title'] + " " + post['text']
        reddit_corpus += preprocess(string_ripped_out)
        
    subreddits_without_titles.append(reddit_corpus)

In [29]:
songs_without_titles = []
for song in songs_list:
    lyrics = song['text']
    songs_without_titles.append(lyrics)

In [74]:
full_docs = [subreddits_without_titles[10]] + songs_without_titles 

In [75]:
tfidf_matrix = tfidf_vectorizer.fit_transform(full_docs)

In [88]:
result = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix)

In [95]:
print(result[0])

temp = result[0].tolist()
temp.remove(max(result[0]))

songs_list[temp.index(max(temp))]

[ 1.          0.03148522  0.00885396 ...,  0.03552177  0.01527526
  0.03386445]


{'artist': 'Alphaville',
 'lyric_link': '/a/alphaville/all+in+the+golden+afternoon_20006851.html',
 'text': "All in the golden afternoon full leisurely we glide  \nFor both our oars, with little skill, by little arms are plied  \nWhile little hands make vain pretence our wanderings to guide  \nAh, cruel three! in such an hour beneath such dreamy wheather  \nTo beg a tale of breath too weak to stir the tiniest feather  \nAnd what can one poor voice avail against three tongues together  \nAnon, to sudden silence won, in fancy they pursue  \nThe dream child moving through a land of wonders wild and new  \n  \nIn friendly chat with bird or beast- and half believe it true  \nAnd ever as the story drained the wells of fancy dry and faintly strove that\nweary one to put the subject by  \nThe next time. it is next time the happy voices cry  \nThus grew the tale of wonderland, thus slowly, one by one  \nIt's quaint events were hammered out  \nAnd now the tale is done and home we steer  \nA merr

In [87]:
print(result)

[[ 1.          0.03148522  0.00885396 ...,  0.03552177  0.01527526
   0.03386445]]
